# Hangman AI using ANN

- This AI will predict the next best possible character given the list of characters already predicted (or) game state.- AI was trained on data on how a good game of Hangman looks like.

## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Creating Dataset

### 1. Importing Word List

In [2]:
f = open("D:/testLab/wordList.txt", "r")
words = []
for i in f.readlines():
    words.append((str(i).lower())[:-1])
f.close()

### 2. Calculating max_word_length 

In [3]:
max_word_length = -1
for word in words:
    if len(word) > max_word_length:
        max_word_length = len(word)

In [4]:
print(max_word_length)

29


### 3. Splitting words for training and testing

In [5]:
training_words, testing_words = train_test_split(words, test_size = 0.2, random_state = 0)

In [6]:
training_words[:5]

['unadhesively', 'timbersome', 'geordie', 'euxine', 'chemar']

### 4. Creating the features and target variable

- Converting each character in each word in integer format
- Adding additional 0's before each word in integer format to make all the words length equal

In [7]:
temp_X = []
t = [] # Tempory List
for i, word in enumerate(words):
    for j in ("0"*(max_word_length - len(word))):
        t.append(j)
    for char in word:
        t.append(str(ord(char) - 96))
    temp_X.append(t)
    t = []

In [8]:
print(temp_X[5])

['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '7']


- Converting to desired input format

In [9]:
y = []
X = []
bigX = []
d = {}
for i in range(len(temp_X)):
    d = {} # Counting frequency of the letters in the word
    for j in temp_X[i]:
        if (j != '0'):
            if j in d:
                d[j] += 1
            else:
                d[j] = 1
    d2 = {} # Sorting the dictionary in descending order based on value pair
    for j in range(len(d)):
        el = ""
        elM = -1
        for k in d:
            if d[k] > elM and k not in d2:
                elM = d[k]
                el = k
        d2[el] = elM
    d = d2
    for j in d:
        newX = [0 for k in range(len(temp_X[i]))]
        for k in d:
            if j == k:
                break
            else:
                for I in range(len(temp_X[i])):
                    if temp_X[i][I] == k:
                        newX[I] = k
        y.append(j)
        newX.insert(0, len(words[i]))
        X.append(newX)

- Converting all datatype from chr to int

In [10]:
new_X = []
for i in X:
    new_X.append(list(map(int, i)))
X = new_X
y = list(map(int, y))

### Explanation:-
max_length_word : cyclotrimethylenetrinitramine

length : 29

-----------
Word : Fatee
-----------
In [7]:-

['0', '0' .... '6', '1', '20', '5', '5'] 

{F: 6, A: 1, T: 20, E: 5}
Total lenth of array => 29
-----------
In [9]:-

d = {'F': 1, 'A': 1, 'T': 1, 'E': 2}

d = {'E': 2, 'F': 1, 'A': 1, 'T': 1}  # Sorted form of d

X = [

[0, 0 .... 0, 0, 0, 0, 0]

[0, 0 .... 0, 0, 0, 5, 5]

[0, 0 .... 6, 0, 0, 5, 5]

[0, 0 .... 6, 1, 0, 5, 5]

]

y = [5, 6, 1, 20]

-----------
1) Feature Scaling

2) Conversion to numpy array

- Converting to numpy array
- Reshaping array

In [11]:
X = np.array(X)
y = np.array(y)
y.reshape((-1, 1))

array([[ 1],
       [ 1],
       [ 1],
       ...,
       [ 1],
       [26],
       [26]])

In [12]:
print(X[5], y[5])
print(X[6], y[6])
print(X[7], y[7])
print(X[8], y[8])
print(X[9], y[9])

[6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0] 3
[6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 3 0 0 0] 8
[6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 3 8 0 0] 5
[6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 3 8 5 0] 14
[4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 1


In [13]:
X_train = X

### 6. Feature Scaling

In [14]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)

### 7. Converting y to categories

In [15]:
y_train = tf.keras.utils.to_categorical(y - 1, num_classes=26)

## Creating the ANN Model

In [16]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=128, activation='relu')) 
ann.add(tf.keras.layers.Dense(units=128, activation='relu')) 
ann.add(tf.keras.layers.Dense(units=128, activation='relu')) 
ann.add(tf.keras.layers.Dense(units=128, activation='relu'))
ann.add(tf.keras.layers.Dense(units=26, activation='softmax'))
ann.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

## Training the Model

In [17]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 10)

Epoch 1/10
52538/52538 [==============================] - 96s 2ms/step - loss: 2.7818 - accuracy: 0.1609
Epoch 2/10
52538/52538 [==============================] - 98s 2ms/step - loss: 2.6756 - accuracy: 0.1913
Epoch 3/10
52538/52538 [==============================] - 79s 2ms/step - loss: 2.6359 - accuracy: 0.2012
Epoch 4/10
52538/52538 [==============================] - 78s 1ms/step - loss: 2.6128 - accuracy: 0.2070
Epoch 5/10
52538/52538 [==============================] - 78s 1ms/step - loss: 2.5976 - accuracy: 0.2111
Epoch 6/10
52538/52538 [==============================] - 78s 1ms/step - loss: 2.5867 - accuracy: 0.2142
Epoch 7/10
52538/52538 [==============================] - 78s 1ms/step - loss: 2.5788 - accuracy: 0.2159
Epoch 8/10
52538/52538 [==============================] - 80s 2ms/step - loss: 2.5727 - accuracy: 0.2176
Epoch 9/10
52538/52538 [==============================] - 80s 2ms/step - loss: 2.5671 - accuracy: 0.2191
Epoch 10/10
52538/52538 [==============================

## Hangman Game:-

In [18]:
class Hangman:
    def __init__(self):
        self.word = random.choice(words)
        self.codedWord = self.convert_to_integer(self.word)
        self.guess = [0 for i in range(max_word_length)]
        self.guess.insert(0, self.codedWord[0])
    
    def reset(self):
        self.word = random.choice(words)
        self.codedWord = self.convert_to_integer(self.word)
        self.guess = [0 for i in range(max_word_length)]
        self.guess.insert(0, self.codedWord[0])
    
    def convert_to_integer(self, word):
        t = []
        for i in ("0"*(max_word_length-len(word))):
            t.append(i)
        for i in word:
            t.append(ord(i)-96)
        t.insert(0, str(len(word)))
        for i in range(len(t)):
            t[i] = int(t[i])
        return t
    
    def feature_scale_word(self, word):
        word = [word]
        word = sc.transform(word)
        return word
    
    def check_guessed_character(self, char):
        if char in self.codedWord:
            for i in range(len(self.guess)):
                if (self.codedWord[i] == char):
                    self.guess[i] = char
            return 1
        else:
            return 0
    
    def human_read_format(self):
        string = ""
        for i in range(len(self.codedWord)-len(self.word), len(self.guess)):
            if self.guess[i] > 0:
                string += chr(self.guess[i]+ord('a')-1)
            else:
                string += " "
        return string
    
    def win(self):
        if self.codedWord == self.guess:
            return 1
        else:
            return 0
        
    def int_to_char(self, char):
        return chr(char+ord('a')-1)

In [19]:
hangman_game = Hangman()

In [20]:
num = 0
while not hangman_game.win():
    num += 1
    print("Try: " + str(num) + " Word: '" + hangman_game.human_read_format()+"'")
    y_pred = ann.predict(hangman_game.feature_scale_word(hangman_game.guess))
    predicted_category = int(tf.argmax(y_pred, axis=1) + 1)
    print("Guessed Character : " + hangman_game.int_to_char(predicted_category))
    hangman_game.check_guessed_character(predicted_category)
    if (num>=50):
        break

Try: 1 Word: '          '
1/1 [==============================] - 0s 76ms/step
Guessed Character : e
Try: 2 Word: '     e    '
1/1 [==============================] - 0s 15ms/step
Guessed Character : e
Try: 3 Word: '     e    '
1/1 [==============================] - 0s 15ms/step
Guessed Character : e
Try: 4 Word: '     e    '
1/1 [==============================] - 0s 16ms/step
Guessed Character : e
Try: 5 Word: '     e    '
1/1 [==============================] - 0s 15ms/step
Guessed Character : e
Try: 6 Word: '     e    '
1/1 [==============================] - 0s 16ms/step
Guessed Character : e
Try: 7 Word: '     e    '
1/1 [==============================] - 0s 17ms/step
Guessed Character : e
Try: 8 Word: '     e    '
1/1 [==============================] - 0s 17ms/step
Guessed Character : e
Try: 9 Word: '     e    '
1/1 [==============================] - 0s 18ms/step
Guessed Character : e
Try: 10 Word: '     e    '
1/1 [==============================] - 0s 16ms/step
Guessed Character : e